In [37]:
import torch
import torch.nn as nn
from torch.nn  import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
blockSize = 8
batchSize = 4
iters = 1000
lr = 3e-4
evali = 250
# dropout = 0.2

cuda


In [38]:
with open ('Gatsby.txt', 'r', encoding='UTF-8') as f:
    text = f.read()

chars=sorted(set(text))
print(chars) 

vocab= len(chars)

['\t', '\n', ' ', '!', '$', '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'ç', 'é', 'ê', 'ô', '\u200a', '—', '‘', '’', '“', '”', '…', '\ufeff']


In [39]:
char_to_int = { ch:i  for i,ch in enumerate(chars)}
int_to_char = { i:ch for i,ch in enumerate(chars)}

encode = lambda s:[char_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_char[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
print(data[:1000])

tensor([89,  1,  1,  0,  0,  0,  2,  2,  2, 43, 59, 56,  2, 30, 69, 56, 52, 71,
         2, 30, 52, 71, 70, 53, 76,  1,  0,  0,  0,  0,  2,  2, 53, 76,  1,  0,
         0,  0,  2, 29, 10,  2, 42, 54, 66, 71, 71,  2, 29, 60, 71, 77, 58, 56,
        69, 52, 63, 55,  1,  1,  1,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
         2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2, 43, 52,
        53, 63, 56,  2, 66, 57,  2, 26, 66, 65, 71, 56, 65, 71, 70,  1,  1, 32,
         1, 32, 32,  1, 32, 32, 32,  1, 32, 45,  1, 45,  1, 45, 32,  1, 45, 32,
        32,  1, 45, 32, 32, 32,  1, 32, 47,  1,  1,  1,  2,  2,  2,  2,  2,  2,
         2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
         2,  2,  2,  2,  2,  2, 38, 65, 54, 56,  2, 52, 58, 52, 60, 65,  1,  2,
         2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
         2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2, 71, 66,  1,
         2,  2,  2,  2,  2,  2,  2,  2, 

In [40]:
n = int(0.8*len(data))
train= data[:n]
val= data[:n]

def getBatch(split):
    data = train if split == 'train' else val
    ix = torch.randint(len(data) - blockSize,(batchSize,) )
    x = torch.stack([data[i:i+blockSize] for i in ix])
    y = torch.stack([data[i+1:i+blockSize+1] for i in ix])
    x,y = x.to(device), y.to(device)
    return x,y

x,y = getBatch('train')

print('inputs:')
print(x)

print('outputs:')
print (y)

inputs:
tensor([[69, 56, 70, 71, 63, 56, 70, 70],
        [72, 71,  2, 60, 71,  2, 57, 66],
        [ 2, 62, 65, 66, 74, 70,  2, 74],
        [69,  8,  2, 61, 72, 70, 71,  2]], device='cuda:0')
outputs:
tensor([[56, 70, 71, 63, 56, 70, 70,  2],
        [71,  2, 60, 71,  2, 57, 66, 69],
        [62, 65, 66, 74, 70,  2, 74, 59],
        [ 8,  2, 61, 72, 70, 71,  2, 52]], device='cuda:0')


In [41]:
x = train[:blockSize]
y = train[1:blockSize+1]

for t in range(blockSize):
    context = x[:t+1]
    target = y[t]
    print(context, 'output:', target)

tensor([89]) output: tensor(1)
tensor([89,  1]) output: tensor(1)
tensor([89,  1,  1]) output: tensor(0)
tensor([89,  1,  1,  0]) output: tensor(0)
tensor([89,  1,  1,  0,  0]) output: tensor(0)
tensor([89,  1,  1,  0,  0,  0]) output: tensor(2)
tensor([89,  1,  1,  0,  0,  0,  2]) output: tensor(2)
tensor([89,  1,  1,  0,  0,  0,  2,  2]) output: tensor(2)


In [47]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(evali)
        for k in range(evali):
            X, Y = getBatch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [48]:
class Bigram(nn.Module):
    
    def __init__(self, vocab):
        super().__init__()
        self.token = nn.Embedding(vocab, vocab)
        
    def forward(self, index, targets=None):
        logits = self.token(index)
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape 
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, index, max_tokens):
        
        for _ in range (max_tokens):
            logits, loss = self.forward(index)
            logits = logits[:,-1,:]
            probs = F.softmax(logits, dim=-1)
            index_next = torch.multinomial(probs, num_samples=1)
            index = torch.cat((index, index_next), dim=1)
            
        return index
    
model = Bigram(vocab)
m = model.to(device)
    
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated = decode(m.generate(context, max_tokens=500)[0].tolist())
print(generated)    
    

	Hd uçCuxKR
ésiuxq2S,﻿S﻿rLUcTFLgS B B2fNLVmM:dreç3DuKj?”“8c“oM	Vzi?xKo$lv!lt:mXSdIkj
ô﻿bXUURS	MVmTulKM?[dBhBéxXi?1dOo8Fshô	zç:SANXOLsqH]!iô],E0Mê4”CdZ-“Kz$“(ipkVUi6	Gzu2.tP—ç 6;avo7W…N6$9’m2f‘ZzcR…UeLfhI(sVDOpOL5(—zme$éXcv
-OcGU;A”EsTêH3]20?R4yv!K1’TsC— wB08or)LK;
éZ—??2TzLFxB,o9c1xjcT4h)ij5.W$nlsc4K129ce !	g0e*m’rLgKnWSX!zfsêq2zbudqcul!zQNRtcv F-6Kh]…Nyvr!KgrsIK’894êHH.JsK2S929qêG FMT6AmLstt5dr,hG *…x*U…Gy	8!(![0eCGQE0wWzNR4rQhV’uEhMhD…I0S
dPAac”I‘$6W“eyO-PY !BD5N’nAd Ea7 Y(b8
ZXU
uxFXcQLêç7;x“g


In [53]:
optimize = torch.optim.AdamW(model.parameters(), lr=lr)

for iter in range(iters):
    if iter % evali == 0:
        losses = estimate_loss()
        print(f"step: {iter}, tloss:{losses['train']}, vloss:{losses['val']}")
        
    xb, yb = getBatch('train')
    
    logits, loss = model.forward(xb, yb)
    optimize.zero_grad(set_to_none=True)
    loss.backward()
    optimize.step()
    
print(loss.item())
        

step: 0, tloss:4.776259899139404, vloss:4.786865234375
step: 250, tloss:4.702925682067871, vloss:4.714049339294434
step: 500, tloss:4.660702705383301, vloss:4.664233207702637
step: 750, tloss:4.600022792816162, vloss:4.599461555480957
4.37638521194458
